In [83]:
!pip install scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 54.1 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)


In [111]:
import random

from pyprojroot import here
from datasets import load_dataset, Dataset
from huggingface_hub import login
from sklearn.model_selection import train_test_split

In [63]:
login()

In [3]:
dataset = "hendrycks/competition_math"

In [4]:
ds = load_dataset(dataset)

/home/ubuntu/jupyter/env/lib/python3.8/site-packages/datasets/load.py:1486: FutureWarning: The repository for hendrycks/competition_math contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hendrycks/competition_math
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/7500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [10]:
ds["test"][5]

{'problem': 'What is the 100th term of the arithmetic sequence 6, 10, 14, 18, ...?',
 'level': 'Level 2',
 'type': 'Algebra',
 'solution': 'The common difference is $10 - 6 = 4$, so the 100th term is $6+99\\cdot 4=\\boxed{402}$.'}

In [11]:
!pip install pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=5d057685ba9657e5acb07d00878b3c1f738820d0f2285ff2460682ae7804efe2
  Stored in directory: /home/ubuntu/.cache/pip/wheels/72/99/be/81d9bcdf5dd5ee5acd8119a9dd5bc07204c9ce205fd341b021
Successfully built pylatexenc

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [35]:
import re

def extract_text_within_boxed(latex_str):
    pattern = r'\\boxed{(.*?)}'
    match = re.search(pattern, latex_str)
    
    if match:
        return match.group(1)
    return None

latex_str = r"The common difference is $10 - 6 = 4$, so the 100th term is $6+99\\cdot 4=\\boxed{402}$."
extracted_text = extract_text_within_boxed(latex_str)
print(extracted_text)  # Output: This is the extracted text

402


In [57]:
ds_subsample = Dataset.from_list(random.sample(list(ds["test"]), 200))

In [54]:
ds["test"][0]

{'problem': 'How many vertical asymptotes does the graph of $y=\\frac{2}{x^2+x-6}$ have?',
 'level': 'Level 3',
 'type': 'Algebra',
 'solution': 'The denominator of the rational function factors into $x^2+x-6=(x-2)(x+3)$. Since the numerator is always nonzero, there is a vertical asymptote whenever the denominator is $0$, which occurs for $x = 2$ and $x = -3$.  Therefore, the graph has $\\boxed{2}$ vertical asymptotes.'}

In [68]:
print(set(ds["test"]["level"]))
print(set(ds["test"]["type"]))

{'Level 1', 'Level 2', 'Level 4', 'Level 5', 'Level 3'}
{'Geometry', 'Algebra', 'Precalculus', 'Counting & Probability', 'Intermediate Algebra', 'Prealgebra', 'Number Theory'}


In [70]:
ds["test"]

Dataset({
    features: ['problem', 'level', 'type', 'solution'],
    num_rows: 5000
})

In [91]:
ds["test"].to_pandas()

,problem,level,type,solution
0,How many vertical asymptotes does the graph of...,Level 3,Algebra,The denominator of the rational function facto...
1,What is the positive difference between $120\%...,Level 1,Algebra,One hundred twenty percent of 30 is $120\cdot3...
2,Find $x$ such that $\lceil x \rceil + x = \dfr...,Level 4,Algebra,"First, we note that $x$ must be positive, sinc..."
3,Evaluate $i^5+i^{-25}+i^{45}$.,Level 5,Algebra,We have $i^5 = i^4\cdot i = 1\cdot (i) = i$. ...
4,"If $2^8=4^x$, what is the value of $x$?",Level 1,Algebra,Rewrite $4$ as $2^2$ to find $4^x=2^{2x}$. Si...
...,...,...,...,...
4995,If $\sin x + \cos x = \frac{1}{5}$ and $0 < x ...,Level 5,Precalculus,"From the given equation, $\cos x = \frac{1}{5}..."
4996,The matrix for projecting onto a certain plane...,Level 5,Precalculus,Since $\begin{pmatrix} a \\ b \\ c \end{pmatri...
4997,"Let $\mathbf{a},$ $\mathbf{b},$ and $\mathbf{c...",Level 4,Precalculus,Since $\mathbf{a} + \mathbf{b} + \mathbf{c} = ...
4998,Find the smallest positive integer solution to...,Level 5,Precalculus,"By the tangent addition formula,\n\begin{align..."


In [92]:
strata = ["level", "type"]

df = ds["test"].to_pandas()

subsample, _ = train_test_split(df, train_size=200, random_state=40, stratify=df[strata])

In [103]:
subsample

,problem,level,type,solution
852,"Suppose $P$ is the point $(5,3)$ and $Q$ is th...",Level 3,Algebra,"Let the coordinates of $T$ be $(x,y)$. Then, t..."
3920,If $4$ wands are equivalent to $6$ rands and $...,Level 4,Prealgebra,We can set up a proportion to find the ratio o...
2180,"Let $a,$ $b,$ $c,$ and $d$ be positive real nu...",Level 5,Intermediate Algebra,"By AM-GM,\n\[\frac{\underbrace{3a + 3a + \dots..."
1963,Altitudes $\overline{AD}$ and $\overline{BE}$ ...,Level 4,Geometry,"First, we build a diagram:\n\n[asy]\n\nsize(15..."
1135,What is the greatest integer value of $x$ for ...,Level 3,Algebra,We can factor $6x^2+x-2$ as $(3x+2)(2x-1)$. Fo...
...,...,...,...,...
4897,"If $\sin^2 \theta + \sin \theta = 1,$ then fin...",Level 2,Precalculus,"From $\sin^2 \theta + \sin \theta = 1,$ $\sin ..."
1828,Compute $\sin 30^\circ$.,Level 1,Geometry,Let $P$ be the point on the unit circle that i...
2128,"In the diagram, square $ABCD$ has sides of len...",Level 5,Geometry,"Since $\triangle ABE$ is equilateral, we have ..."
596,Evaluate $\lceil (3.6)^2 \rceil - ( \lceil 3.6...,Level 4,Algebra,$\lceil (3.6)^2 \rceil = \lceil 12.96 \rceil =...


In [97]:
ds_subsample = Dataset.from_pandas(subsample)

In [104]:
ds_subsample = ds_subsample.remove_columns("__index_level_0__")
ds_subsample

Dataset({
    features: ['problem', 'level', 'type', 'solution'],
    num_rows: 200
})

In [105]:
ds_subsample.push_to_hub("sachithgunasekara/self-discover-MATH-subsample")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/376 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/sachithgunasekara/self-discover-MATH-subsample/commit/d0e03c6022cd801cd984c3fda3d51302ffec0064', commit_message='Upload dataset', commit_description='', oid='d0e03c6022cd801cd984c3fda3d51302ffec0064', pr_url=None, pr_revision=None, pr_num=None)

In [109]:
random.sample(list(load_dataset("hendrycks/competition_math", split="train").filter(lambda x: x["level"] == level and x["type"] == type)), 1)[0]["problem"]

'What is the base-10 integer 515 when expressed in base 6?'

In [113]:
ds_subsample.to_csv(here("hendrycks_competition_math_subsample.csv"))

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

140012

In [122]:
from pylatexenc.latex2text import LatexNodes2Text

def to_html(instance):
    return {
        "problem_html": LatexNodes2Text().latex_to_text(instance["problem"]),
        "solution_html": LatexNodes2Text().latex_to_text(instance["solution"])
    }

csv_subsam = ds_subsample.map(to_html)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [124]:
csv_subsam[0]["solution_html"]

'Let the coordinates of T be (x,y). Then, the midpoint of PT is (x+5/2, y+3/2).  Since we know the coordinates of this point are those of point Q, (-3,6), we have (x+5)/2 = -3 and (y+3)/2 = 6.  Solving these equations gives x = -11 and y = 9, so point T is at (-11,9).'

In [123]:
csv_subsam.to_csv(here("hendrycks_competition_math_subsample.csv"))

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

258405